## Graph construction using networkx and graph-tool

In [3]:
import networkx as nx
import numpy as np
from itertools import product
import numpy.ma as ma
from toolz.curried import pipe, curry
from toolz.curried import map as fmap
import zipfile
import pandas
import time

from graph_tool.all import *
from graph_tool.topology import mark_subgraph
from graph_tool.centrality import betweenness

Import data files 

In [5]:
zip_stream = zipfile.ZipFile('../data/cahn-hilliard.zip', 'r')

def read_data(file_name):
    return np.array(
        pandas.read_csv(
            zip_stream.open(file_name, 'r'),
            delimiter=' ',
            header=None               
        ).swapaxes(0, 1)
    )

data = np.array(
    list(
        map(read_data, zip_stream.namelist()[1:6])
    )
)

In [6]:
cond = lambda x: (x[:, 1] != -1) & (x[:, 0] != -1)


def merge_edges(neighbors, ids):
    return pipe(
        ids,
        lambda x: np.reshape(x.flatten(), (-1, 1, 1)),
        lambda x: np.repeat(x, neighbors.shape[1], axis=1),
        lambda x: np.concatenate((x, neighbors), axis=-1).reshape(-1, 2),
        lambda x: x[cond(x)],
    )


def index_vectors(nx, ny, nz):
    index2D = ([1, 0, 0], [1, 1, 0], [0, 1, 0], [-1, 1, 0])
    if ny == nz == 1:
        return ([1, 0, 0],)
    if nz == 1:
        return index2D
    else:
        return index2D + tuple([x, y, 1] for x in (1, 0, -1) for y in (1, 0, -1))


def make_ids_padded(ids):
    nx, ny, nz = ids.shape
    ids_padded = -np.ones((nx + 2, ny + 2, nz + 2), dtype=int)
    ids_padded[1:-1, 1:-1, 1:-1] = ids
    return ids_padded


@curry
def make_sub_ids(ids_padded, indices):
    nx, ny, nz = ids_padded.shape
    left, up, front = indices
    return ids_padded[
        1 + left : nx - 1 + left, 1 + up : ny - 1 + up, 1 + front : nz - 1 + front
    ][..., None]


def make_neighbors(ids):
    nx, ny, nz = ids.shape
    ids_padded = make_ids_padded(ids)

    return pipe(
        index_vectors(nx, ny, nz),
        fmap(make_sub_ids(ids_padded)),
        list,
        lambda x: np.concatenate(x, axis=-1).reshape(nx * ny * nz, len(x), 1),
    )


def make_grid_edges(nx=1, ny=1, nz=1):
    ids = np.arange(nx * ny * nz).reshape(nx, ny, nz)
    return merge_edges(make_neighbors(ids), ids)


def make_grid_graph(shape):
    g = nx.Graph()
    g.add_nodes_from(np.arange(np.prod(shape)))
    g.add_edges_from(make_grid_edges(*shape))
    return g

def make_grid_graph_gt(shape):
    g = Graph(directed = False)
    g.add_vertex(np.prod(shape))
    g.add_edge_list(make_grid_edges(*shape))
    return g


In [7]:
start_time = time.time()
G = make_grid_graph_gt(data[0].shape)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.03610086441040039 seconds ---


In [48]:
start_time = time.time()
g = make_grid_graph(data.shape)
print("--- %s seconds ---" % (time.time() - start_time))

--- 7.095793962478638 seconds ---


In [8]:
v = G.get_vertices() 
vprop_double = G.new_vertex_property("int")
v

array([    0,     1,     2, ..., 40498, 40499, 40500])

In [9]:
vertex_colors = data[0].flatten()
vertex_colors.shape

(40501,)

In [10]:
for i in range(len(vertex_colors)):
    vprop_double[i] = vertex_colors[i]
G.vertex_properties["color"] = vprop_double

In [11]:
phases = G.vertex_properties["color"]
interface = []
for n, u in G.edges():
    if phases[n] != phases[u]:
        interface.append([n,u])

In [ ]:
bv, be = betweenness(G)
be.a /= be.a.max() / 5
graph_draw(g, pos=pos, vertex_fill_color=bv, edge_pen_width=be,
           output="filtered-bt.svg")

In [ ]:
for e in G.edges():
    if phases[e.source()] != phases[e.target()]:
        G.remove_edge(e)

In [63]:
phase = G.vertex_properties["color"]

np.array(list(phases))
phases = G.vertex_properties["color"]
interface = [
    (n, u)
    for n, u in G.edges()
    if phases[n] != phases[u]
]

TypeError: list expected at most 1 argument, got 2

In [64]:
list(interface)

[(<Vertex object with index '8' at 0x1920469c0>,
  <Vertex object with index '110' at 0x192046bc0>),
 (<Vertex object with index '8' at 0x1920467c0>,
  <Vertex object with index '9' at 0x192046940>),
 (<Vertex object with index '35' at 0x192046dc0>,
  <Vertex object with index '137' at 0x192064040>),
 (<Vertex object with index '35' at 0x1920468c0>,
  <Vertex object with index '36' at 0x192046a40>),
 (<Vertex object with index '49' at 0x1920640c0>,
  <Vertex object with index '151' at 0x192064140>),
 (<Vertex object with index '49' at 0x1920641c0>,
  <Vertex object with index '50' at 0x192064240>),
 (<Vertex object with index '69' at 0x192064340>,
  <Vertex object with index '171' at 0x1920642c0>),
 (<Vertex object with index '69' at 0x192064440>,
  <Vertex object with index '70' at 0x1920643c0>),
 (<Vertex object with index '109' at 0x1920644c0>,
  <Vertex object with index '211' at 0x192064540>),
 (<Vertex object with index '109' at 0x192064640>,
  <Vertex object with index '110' at 

In [58]:
def makeImageGraph(morph):

    G = make_grid_graph(morph.shape)
    vertex_colors = morph.flatten()
    mapping = {(i): vertex_colors[i] for i in range(len(vertex_colors))}
    nx.set_node_attributes(G, mapping, name="color")
    return G

def makeImageGraph_gt(morph):

    G = make_grid_graph(morph.shape)
    v = G.get_vertices() 
    vphase = G.new_vertex_property("int")
    vertex_colors = data[0].flatten()
    for i in range(len(vertex_colors)):
        vphase[i] = vertex_colors[i]
    G.vertex_properties["color"] = vphase
    return G


def count_of_vertices(G, phase):
    
    phases = nx.get_node_attributes(G, "color")
    phase_list = list(phases.values())
    return phase_list.count(phase)

def count_of_vertices_gt(G, phase):
    phases = np.array(list(G.vertex_properties["color"]))
    phasec = (phases == phase).sum()
    return phasec

def node_phaseA(n, G):
    nodes = G.nodes
    return nodes[n]["color"] == 0


def node_phaseB(n, G):
    nodes = G.nodes
    return nodes[n]["color"] == 1


def makeInterfaceEdges(G):
    
    interface = [
        (n, u)
        for e, i in G.edges()
        if (node_phaseA(n, G) and node_phaseB(u, G))
        or (node_phaseB(n, G) and node_phaseA(u, G))
    ]
    G.remove_edges_from(interface)
    G.add_node(-1, color="green")
    interface = np.unique(np.array(interface))
    interface_edges = [(x, -1) for x in interface]
    G.add_edges_from(interface_edges)
    return G

def makeInterfaceEdges_gt(G):
    phases = G.vertex_properties["color"]
    interface = [
        (n, u)
        for n, u in G.edges()
        if phases[n] != phases[u]
    ]
    for i in interface:
        print(i)
        G.remove_edge(i)
    G.add_node(-1, color="green")
    interface = np.unique(np.array(interface))
    interface_edges = [(x, -1) for x in interface]
    G.add_edge_list(interface_edges)
    return G

def makeConnectedComponents(G, phase):

    nodes = (node for node, data in G.nodes(data=True) if data.get("color") == phase)
    subgraph = G.subgraph(nodes)
    subgraph.nodes
    return nx.number_connected_components(subgraph)


def interfaceArea(G):
    
    nodes_0 = [
        neighbor for neighbor in G.neighbors(-1) if G.nodes[neighbor]["color"] == 0
    ]
    nodes_1 = [
        neighbor for neighbor in G.neighbors(-1) if G.nodes[neighbor]["color"] == 1
    ]
    return G.degree[-1], len(nodes_0), len(nodes_1)


def shortest_distances_all(G):
    
    path = nx.single_source_shortest_path(G, -1)
    del path[-1]
    path_length = [len(p) for p in path.values()]
    # print(path_length)
    return sum(path_length) / len(path_length)


def shortest_distances_phase(G, phase):
   
    source = [node for node, data in G.nodes(data=True) if data.get("color") == phase]
    path = [
        nx.shortest_path(G, s, target=-1, weight=None, method="dijkstra")
        for s in source
    ]
    path_length = [len(p) for p in path]
    return sum(path_length) / len(path_length)


def shortest_dist_boundary(G, phase):
    path = nx.single_source_shortest_path(g, -1)
    path_length = [len(p) for p in path.values()]
    return sum(path_length) / len(path_length)


def tortuosity(G, phase):
    return None


def interface_boundary(G, phase):
    return None


def getGraspiDescriptors(data):
   
    

    g = makeImageGraph(data)
    
   # g = makeInterfaceEdges(g)
   # [interface_area, phase_0_interface, phase_1_interface] = interfaceArea(g)

#    return dict(
#        phase_0_count=count_of_vertices(g, 0),
#        phase_1_count=count_of_vertices(g, 1),
#        phase_0_cc=makeConnectedComponents(g, 0),
#        phase_1_cc=makeConnectedComponents(g, 1),
#        interfacial_area=interface_area,
#        phase_0_interface=phase_0_interface,
#        phase_1_interface=phase_1_interface,
#        distance_to_interface=shortest_distances_all(g),
#        distance_to_interface_0=shortest_distances_phase(g, 0),
#        distance_to_interface_1=shortest_distances_phase(g, 1),
#    )


In [20]:
g = complete_graph(30)
sub = complete_graph(10)

<Graph object, undirected, with 10 vertices and 45 edges, at 0x1622c6280>